In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path
from pathlib import Path
import numpy as np
import pandas as pd
import os # For path.basename and cpu_count
import tensorflow as tf # Ensure tf is imported for type hints and operations
from tensorflow.keras.models import load_model # For loading model in worker
import concurrent.futures
from tqdm import tqdm # For progress bar

In [2]:
def combined_mse_cosine_loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    y_true_norm = tf.nn.l2_normalize(y_true, axis=1)
    y_pred_norm = tf.nn.l2_normalize(y_pred, axis=1)
    cosine_loss = 1 - tf.reduce_mean(tf.reduce_sum(y_true_norm * y_pred_norm, axis=1))
    return mse + 0.3 * cosine_loss

In [3]:
model_path = "models/custom_cnn.keras"
new_spectrogram_path = "spectrogram/test_data/000002_spectrogram_win_length=2048_hop_length=512_n_fft=2048.png"
FEATURE_NAMES = [
    "acousticness", "instrumentalness", "liveness", "speechiness",
    "danceability", "energy", "tempo", "valence"
]

if len(FEATURE_NAMES) != 8:
    raise ValueError("FEATURE_NAMES list must contain exactly 8 names.")

In [4]:
# 1. Load the trained model with the custom loss function
print(f"Loading model from: {model_path}")
# Ensure model_path is a string or Path object correctly pointing to your model
# from pathlib import Path # if you want to use Path objects
# model_path_obj = Path(model_path)
# if not model_path_obj.exists():
#     raise FileNotFoundError(f"Model file not found at {model_path}")

try:
    # It's good practice to clear session in notebooks if re-running model related code
    tf.keras.backend.clear_session()

    model = load_model(
        model_path,
        custom_objects={'combined_mse_cosine_loss': combined_mse_cosine_loss}
    )
    print("Model loaded successfully.")
    model.summary()  # Optional: print model summary
except Exception as e:
    print(f"Error loading model: {e}")
    # In a notebook, you might want to raise the exception to stop execution
    raise

Loading model from: models/custom_cnn.keras


I0000 00:00:1747269192.436242  610385 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


Model loaded successfully.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 326, 793, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 163, 396, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 161, 394, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 80, 197, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 78, 195, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 306,074 (1.17 MB)

 Trainable params: 102,024 (398.53 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 204,050 (797.07 KB)

In [5]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (int(984 / 3), int(2385 / 3)))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    image = tf.expand_dims(image, axis=0)
    return image

In [6]:
# Load image
image = load_image(new_spectrogram_path)

# Make predictions
print("\nMaking prediction...")
try:
    with tf.device('/device:GPU:0'):
        predictions = model.predict(image)

    # predictions will be a numpy array like [[feat1, feat2, ..., feat8]]
    predicted_features = predictions[0]

    # Print the results
    print(f"\nPredicted Audio Features for {new_spectrogram_path}:")
    if len(predicted_features) == len(FEATURE_NAMES):
        for name, value in zip(FEATURE_NAMES, predicted_features):
            print(f"- {name}: {value:.4f}")
    else:
        print("Warning: Number of predicted features does not match FEATURE_NAMES length.")
        print("Raw predictions:", predicted_features)

except Exception as e:
    print(f"Error during prediction: {e}")
    # In a notebook, you might want to raise the exception
    raise


Making prediction...


I0000 00:00:1747269193.483968  610528 service.cc:152] XLA service 0x7f52e4006a50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747269193.483998  610528 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-05-14 20:33:13.490356: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747269193.508983  610528 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-14 20:33:14.687749: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng11{k2=2,k3=0} for conv %cudnn-conv-bias-activation.9 = (f32[1,32,326,793]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,328,795]{3,2,1,0} %bitcast.204, f32[32,3,3,3]{3,2,1,0} %bitcast.211, f32[32]{0} %bitcast.213), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op

1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step

Predicted Audio Features for spectrogram/test_data/000002_spectrogram_win_length=2048_hop_length=512_n_fft=2048.png:
- acousticness: 0.4835
- instrumentalness: 0.5398
- liveness: 0.5232
- speechiness: 0.3900
- danceability: 0.1837
- energy: 0.2060
- tempo: 0.4148
- valence: 0.5676


I0000 00:00:1747269215.307757  610528 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [12]:
# --- Configuration for the song database ---
DATABASE_SPECTROGRAM_DIR = Path("spectrogram/fma_large/") # Directory of spectrograms
CSV_FEATURES_PATH = Path("data/echonest_norm.csv") # Path to your CSV with pre-computed features
NUM_SIMILAR_SONGS_TO_FIND = 5


# --- Helper Functions ---
# parse_track_id_from_filename is no longer needed if we only use the CSV

def build_feature_database_from_csv(csv_file_path, feature_names_ordered_list):
    """
    Builds a feature database by reading pre-computed features directly from a CSV file.
    Concise version with fewer sanity checks.
    """
    feature_db = []

    try:
        features_df = pd.read_csv(csv_file_path)
        # Assume the first column is the track_id, rename if not already 'track_id'
        id_column_name = features_df.columns[0]
        if id_column_name != 'track_id':
            features_df.rename(columns={id_column_name: 'track_id'}, inplace=True)
        # features_df.set_index('track_id', inplace=True) # No longer setting index, will iterate rows
    except Exception as e:
        print(f"Error loading or processing CSV file {csv_file_path}: {e}")
        return feature_db # Return empty if CSV loading fails

    print(f"Building database directly from CSV: {csv_file_path}")
    # Iterate over rows of the DataFrame
    for index, row in tqdm(features_df.iterrows(), total=features_df.shape[0], desc="Building database from CSV rows"):
        track_id = row['track_id'] # Get track_id from the row
        try:
            # Select features from the row IN THE ORDER SPECIFIED BY feature_names_ordered_list
            feature_vector = row[feature_names_ordered_list].values.astype(np.float32)
            # Assuming feature_vector will have correct length if all columns in feature_names_ordered_list exist

            # Store the track_id (or other identifier from CSV) and its features
            # Using track_id as the identifier now, not a file path.
            feature_db.append((track_id, feature_vector))
        except (KeyError, ValueError, TypeError) as e:
            print(f"Skipping track_id {track_id} due to error: {e}")
            # Skip this track if features can't be correctly extracted or cast
            continue

    print(f"Feature database built with {len(feature_db)} songs from CSV.")
    return feature_db

def euclidean_distance(vec1, vec2):
    """Computes the Euclidean distance between two vectors."""
    return np.sqrt(np.sum((np.array(vec1) - np.array(vec2))**2))

def find_k_nearest_neighbors(input_song_features, db_features_list, k=5):
    """
    Finds the k most similar songs from the db_features_list to the input_song_features.
    """
    if not db_features_list: # Keep this check as it's fundamental
        return []
    distances = []
    # song_id here will now be the track_id from the CSV
    for song_id, feature_vec in tqdm(db_features_list, desc=f"Finding {k} nearest neighbors"):
        dist = euclidean_distance(input_song_features, feature_vec)
        distances.append((song_id, dist))
    distances.sort(key=lambda item: item[1])
    return distances[:k]

In [13]:
# --- Main Logic for Similarity Search ---

# The following variables are assumed to be defined in your notebook from previous cells:
# - predicted_features: Numpy array of features for the INPUT song (from Keras model).
# - new_spectrogram_path: Path (string or Path object) to the INPUT song's spectrogram (used for display).
# - FEATURE_NAMES: List of 8 feature names in the order your Keras model outputs them.
#   (This is now also used by build_feature_database_from_csv)

# The Keras 'model' object and 'model_path' string are no longer directly used by
# this specific cell's database building logic, but 'model' was used to get 'predicted_features'.

# Assuming FEATURE_NAMES is correctly defined and available from previous cells.
song_feature_database = build_feature_database_from_csv(
    CSV_FEATURES_PATH,
    FEATURE_NAMES # Pass the ordered list of feature names
)

Building database directly from CSV: data/echonest_norm.csv


Building database from CSV rows: 100%|██████████| 13131/13131 [00:01<00:00, 8079.86it/s]

Feature database built with 13131 songs from CSV.


In [15]:
if not song_feature_database:
    print("Cannot perform similarity search because the feature database is empty or could not be built from CSV.")
else:
    display_path_for_input_song = str(new_spectrogram_path) # Used for display only

    print(f"\nFinding {NUM_SIMILAR_SONGS_TO_FIND} songs most similar to '{display_path_for_input_song}'...")
    # The note about matching spectrogram filename is less relevant now, as matching is based on CSV content.
    # However, if your input song's track_id (derived from its filename) happens to be in the CSV,
    # it might still match itself if its features are identical.
    print("Note: The input song itself may appear in results if its features are identical to an entry in the CSV.")

    similar_songs = find_k_nearest_neighbors(
        predicted_features,
        song_feature_database,
        k=NUM_SIMILAR_SONGS_TO_FIND
    )

    if similar_songs:
        print(f"\nTop {len(similar_songs)} similar songs (Track IDs from CSV):")
        for i, (song_id, dist) in enumerate(similar_songs): # song_id is now the track_id
            print(f"{i+1}. Track ID: {int(song_id)} (Distance: {dist:.4f})")
    else:
        print("No similar songs found (this usually means the database was empty or no matches were found).")




Finding 5 songs most similar to 'spectrogram/test_data/000002_spectrogram_win_length=2048_hop_length=512_n_fft=2048.png'...
Note: The input song itself may appear in results if its features are identical to an entry in the CSV.


Finding 5 nearest neighbors: 100%|██████████| 13131/13131 [00:00<00:00, 347272.95it/s]


Top 5 similar songs (Track IDs from CSV):
1. Track ID: 9546 (Distance: 0.4618)
2. Track ID: 16232 (Distance: 0.4887)
3. Track ID: 44249 (Distance: 0.4903)
4. Track ID: 32094 (Distance: 0.4959)
5. Track ID: 45462 (Distance: 0.5114)
